In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [2]:
# Reading data, data comes in 3 separate CSV files, one for each year 2013-2015
data2013 = pd.read_csv('data/MUP_PHY_R19_P04_V10_D13_Prov_Svc.csv')
data2014 = pd.read_csv('data/MUP_PHY_R19_P04_V10_D14_Prov_Svc.csv')
data2015 = pd.read_csv('data/MUP_PHY_R19_P04_V10_D15_Prov_Svc.csv')

C:\Users\Darshan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# Keeping only columns that are needed
data2013 = data2013[['Rndrng_NPI','Rndrng_Prvdr_Type','Rndrng_Prvdr_Gndr','Rndrng_Prvdr_State_Abrvtn','HCPCS_Cd','Tot_Srvcs','Tot_Benes','Tot_Bene_Day_Srvcs','Avg_Sbmtd_Chrg','Avg_Mdcr_Pymt_Amt']]
data2014 = data2014[['Rndrng_NPI','Rndrng_Prvdr_Type','Rndrng_Prvdr_Gndr','Rndrng_Prvdr_State_Abrvtn','HCPCS_Cd','Tot_Srvcs','Tot_Benes','Tot_Bene_Day_Srvcs','Avg_Sbmtd_Chrg','Avg_Mdcr_Pymt_Amt']]
data2015 = data2015[['Rndrng_NPI','Rndrng_Prvdr_Type','Rndrng_Prvdr_Gndr','Rndrng_Prvdr_State_Abrvtn','HCPCS_Cd','Tot_Srvcs','Tot_Benes','Tot_Bene_Day_Srvcs','Avg_Sbmtd_Chrg','Avg_Mdcr_Pymt_Amt']]

In [4]:
# Reading LEIE data
LEIE = pd.read_csv('data/UPDATED.csv')

C:\Users\Darshan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
# Adding the year to each dataset in order to determine fraud
data2013['Year'] = 2013
data2014['Year'] = 2014
data2015['Year'] = 2015

In [6]:
# Preprocessing the LEIE Data

# Removing unused columns
LEIE = LEIE[['NPI', 'EXCLTYPE', 'EXCLDATE', 'REINDATE', 'WAIVERDATE', 'WVRSTATE']]

# Strip the Exclusion type due to excess whitespace
LEIE['EXCLTYPE'] = LEIE['EXCLTYPE'].apply(lambda x: x.strip())

# Data of minimum exclusion period for each Exclusion type
dic = {"1128a1":5,'1128a2':5,"1128a3":5,'1128a4':5,"1128b1":3,'1128b2':3,'1128b3':3,'1128b6':1, '1156':1}
# https://web.archive.org/web/20200522223236/https://oig.hhs.gov/exclusions/authorities.asp

# Adding data of Min exclusion period
LEIE['MINEX'] = LEIE['EXCLTYPE'].apply(lambda x: dic[x] if (x in dic.keys()) else None)

# Removing data of other Exclusion types
LEIE = LEIE[LEIE['MINEX'].notnull()]

# Extracting year from the date of exclusion for data
LEIE['EXCLYR'] = LEIE['EXCLDATE'].apply(lambda x: int(str(x)[:4]))
LEIE['WAIVERYR'] = LEIE['WAIVERDATE'].apply(lambda x: int(str(x)[:4]) if x != 0 else 3000)

LEIE

,NPI,EXCLTYPE,EXCLDATE,REINDATE,WAIVERDATE,WVRSTATE,MINEX,EXCLYR,WAIVERYR
0,0,1128a1,20200319,0,0,NaN,5.0,2020,3000
1,0,1128a1,19880830,0,0,NaN,5.0,1988,3000
3,1922348218,1128a1,20180419,0,0,NaN,5.0,2018,3000
5,0,1128a1,19940524,0,0,NaN,5.0,1994,3000
7,0,1128a1,20110818,0,0,NaN,5.0,2011,3000
...,...,...,...,...,...,...,...,...,...
73093,0,1128a1,20020718,0,0,NaN,5.0,2002,3000
73094,0,1128a1,20160519,0,0,NaN,5.0,2016,3000
73098,0,1128a1,20010118,0,0,NaN,5.0,2001,3000
73101,0,1128a1,20160519,0,0,NaN,5.0,2016,3000


In [7]:
# Adding LEIE data
data2015 = data2015.merge(LEIE, how='left', left_on='Rndrng_NPI', right_on='NPI')

In [8]:
data2014 = data2014.merge(LEIE, how='left', left_on='Rndrng_NPI', right_on='NPI')

In [9]:
data2013 = data2013.merge(LEIE, how='left', left_on='Rndrng_NPI', right_on='NPI')

In [10]:
# Determining if an entry is fraud or not
def fraud(x):
    return 1 if (x['Year'] < (x['MINEX'] + x['EXCLYR'])) and (x['Year'] < x['WAIVERYR']) else 0

data2015['Fraud'] = data2015.apply(fraud, axis=1)
data2014['Fraud'] = data2014.apply(fraud, axis=1)
data2013['Fraud'] = data2013.apply(fraud, axis=1)


In [11]:
# Combining data for all 3 years
data = pd.concat([data2013,data2014,data2015])

In [12]:
# Keeping only columns that are needed
data = data[['Rndrng_NPI','Rndrng_Prvdr_Type','Rndrng_Prvdr_Gndr','Rndrng_Prvdr_State_Abrvtn','HCPCS_Cd','Tot_Srvcs','Tot_Benes','Tot_Bene_Day_Srvcs','Avg_Sbmtd_Chrg','Avg_Mdcr_Pymt_Amt','Fraud']]

In [13]:
# Checking Shape of data
data_fraud = data[data['Fraud'] == 1]
data_nonfraud = data[data['Fraud'] == 0]
(data.shape,data_fraud.shape,data_nonfraud.shape)

((28099708, 11), (22106, 11), (28077602, 11))

In [14]:
# Stratified Sampling of dataset to obrain new dataset
fraud_ratio = 0.00084
total_num = 750000
num_Fraud = int(fraud_ratio*total_num)
df_temp = data_fraud.sample(replace=False, n=num_Fraud, random_state=4012)
df_temp2 = data_nonfraud.sample(replace=False, n=(total_num - num_Fraud), random_state=2104)

In [15]:
# Combining data obtained
data_sampled = pd.concat([df_temp,df_temp2])
X = data_sampled[['Rndrng_NPI','Rndrng_Prvdr_Type','Rndrng_Prvdr_Gndr','Rndrng_Prvdr_State_Abrvtn','HCPCS_Cd','Tot_Srvcs','Tot_Benes','Tot_Bene_Day_Srvcs','Avg_Sbmtd_Chrg','Avg_Mdcr_Pymt_Amt']]
y = data_sampled['Fraud']

# Saving to csv
data_sampled.to_csv('data/data_sampled.csv')

In [30]:
# Obtaining 5 Fold Train and test sets
skf = StratifiedKFold(n_splits=5, random_state=4012, shuffle=True)
i=0
for train_index, test_index in skf.split(X, y):
    i+=1
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train["Fraud"] = y_train
    X_test["Fraud"] = y_test
    print(X_train.Fraud.value_counts())
    print(X_test.Fraud.value_counts())
    X_train.to_csv(f'data/train{i}.csv')
    X_test.to_csv(f'data/test{i}.csv')


TRAIN: [     0      3      5 ... 749997 749998 749999] TEST: [     1      2      4 ... 749985 749990 749996]
0    599496
1       504
Name: Fraud, dtype: int64
0    149874
1       126
Name: Fraud, dtype: int64


<ipython-input-30-2d30843a273e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["Fraud"] = y_train
<ipython-input-30-2d30843a273e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test["Fraud"] = y_test


TRAIN: [     0      1      2 ... 749996 749997 749998] TEST: [     8      9     19 ... 749984 749991 749999]
0    599496
1       504
Name: Fraud, dtype: int64
0    149874
1       126
Name: Fraud, dtype: int64
TRAIN: [     1      2      3 ... 749997 749998 749999] TEST: [     0      5     10 ... 749987 749992 749993]
0    599496
1       504
Name: Fraud, dtype: int64
0    149874
1       126
Name: Fraud, dtype: int64
TRAIN: [     0      1      2 ... 749996 749997 749999] TEST: [    13     16     17 ... 749988 749995 749998]
0    599496
1       504
Name: Fraud, dtype: int64
0    149874
1       126
Name: Fraud, dtype: int64
TRAIN: [     0      1      2 ... 749996 749998 749999] TEST: [     3      6      7 ... 749989 749994 749997]
0    599496
1       504
Name: Fraud, dtype: int64
0    149874
1       126
Name: Fraud, dtype: int64
